In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import collections
from collections import OrderedDict

In [ ]:
infile_test = 'vector_add_dataset_test.npz'
infile_train = 'vector_add_dataset.npz'

In [ ]:
npfile_test = np.load(infile_test)
inputs_test = npfile_test['inputs']
outputs_test = npfile_test['outputs']

npfile_train = np.load(infile_train)
inputs_train = npfile_train['inputs']
outputs_train = npfile_train['outputs']

In [ ]:
model = torch.load('v_add_model')

## Test with a set of vectors

In [ ]:
# standardize the input vectors
inputMeans = inputs_train[0:int(inputs_train.shape[0]),:].mean(axis=0)
inputStdDevs = inputs_train[0:int(inputs_train.shape[0]),:].std(axis=0)
testset_np = (inputs_test-inputMeans)/inputStdDevs
testset = torch.tensor(testset_np).float()

outputMeans = outputs[0:int(outputs.shape[0]),:].mean(axis=0)
outputStdDevs = outputs[0:int(outputs.shape[0]),:].std(axis=0)

# put standardized input vectors through model
results = []
targets = []
errors = []

for i in range(len(testset)):
    # get model output
    model.eval()
    test_output = model(testset[i])
    test_output = test_output.detach().numpy()
    
    # unstandardize output
    test_result = (test_output*outputStdDevs)+outputMeans
    results.append(test_result)
    
    # get target result
    targ_vec = outputs_test[i]
    targets.append(targ_vec)
    
    # get accuracy
    diff = np.linalg.norm(targ_vec - test_result)
    mag = np.linalg.norm(targ_vec)
    error = (diff/mag)*100
    errors.append(error)

In [ ]:
np.mean(errors)

## Test with one vector

In [ ]:
# testing the model with a chosen vector
vector1 = np.array([.50,.50,.50])
vector2 = np.array([.50,.50,.50])

testvec = np.append(vector1,vector2)

# standardize the input vector
inputMeans = inputs_train[0:int(inputs_train.shape[0]),:].mean(axis=0)
inputStdDevs = inputs_train[0:int(inputs_train.shape[0]),:].std(axis=0)
test_input = (testvec-inputMeans)/inputStdDevs
test_input = torch.tensor(test_input).float()

# put standardized input vector through model
model.eval()
test_output = model(test_input)
test_output = test_output.detach().numpy()

# unstandardizing the output
outputMeans = outputs_train[0:int(outputs_train.shape[0]),:].mean(axis=0)
outputStdDevs = outputs_train[0:int(outputs_train.shape[0]),:].std(axis=0)
    
test_result = (test_output*outputStdDevs)+outputMeans

print('model output = {}'.format(test_result))

# compare to target output
x_targ = vector1[0] + vector2[0]
y_targ = vector1[1] + vector2[1]
z_targ = vector1[2] + vector2[2]

targ_vec = np.array([x_targ, y_targ, z_targ])

print('target output = {}'.format(targ_vec))

# measure accuracy
diff = np.linalg.norm(targ_vec - test_result)
mag = np.linalg.norm(targ_vec)
error = (diff/mag)*100
acc = 100 - error
print('error = {}%'.format(error))
print('accuracy = {}%'.format(acc))